<a href="https://colab.research.google.com/github/satuelisa/NLPF/blob/main/NLPF_05_P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gutenbergpy # do not redo this if on your own computer, we already used it in Session 1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for httpsproxy-urllib2: filename=httpsproxy_urllib2-1.0-py3-none-any.whl size=29269 sha256=916672b2b7a8f4dedb475ff574d989cec60e76be1c298987c11755637643639a
  Stored in directory: /root/.cache/pip/wheels/91/04/37/b7080e0cfe33a8bc79ff0082159cea767f9145a5a713b501a3
Successfully built httpsproxy-urllib2


In [38]:
import gutenbergpy.textget # reuse Session 1 stuff again
target = 36 # this one is The War of the Worlds by H.G. Wells
raw  = gutenbergpy.textget.get_text_by_id(target) # content
text = gutenbergpy.textget.strip_headers(raw) # remove header
s = text.decode("utf-8") # string

In [39]:
l = len(s)
margin = 500
midpoint = s[(l - margin):(l + margin)]
print(midpoint)

 vanishing at last into the vague lower sky, to see the people
walking to and fro among the flower beds on the hill, to see the
sight-seers about the Martian machine that stands there still, to hear
the tumult of playing children, and to recall the time when I saw it
all bright and clear-cut, hard and silent, under the dawn of that last
great day. . . .

And strangest of all is it to hold my wife’s hand again, and to think
that I have counted her, and that she has counted me, among the dead.






OK, we have some ideas for words now. Let's see what we can dig up regarding *tumult* in `WordNet`.

In [12]:
import nltk # this we already have
nltk.download('wordnet') # this is new, download once per environment
nltk.download('omw-1.4') # same here

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [13]:
word = 'tumult'
from nltk.corpus import wordnet as wn
ss = wn.synsets(word)
print(ss)

[Synset('tumult.n.01'), Synset('tumult.n.02'), Synset('commotion.n.02')]


What words do these "sets of meaning" contain?

In [15]:
for batch in ss:
  print(batch.lemma_names())

['tumult', 'tumultuousness', 'uproar', 'garboil']
['tumult', 'turmoil']
['commotion', 'din', 'ruction', 'ruckus', 'rumpus', 'tumult']


What is the *meaning* of these?

In [16]:
for batch in ss:
  print(batch.definition())

a state of commotion and noise and confusion
violent agitation
the act of making a noisy disturbance


Let's focus on the last one: *commotion* that means "the act of making a noisy disturbance". What words would be more specific versions of this concepts?

In [17]:
example = ss[-1]
print(example.hyponyms())

[Synset('bustle.n.01')]


How about more general terms for this type of racket?

In [18]:
print(example.hypernyms())

[Synset('disturbance.n.05')]


Let's try another word, "machine". What are possible *components* of a machine?

In [25]:
w2 = 'machine'
s2 = wn.synsets(w2)
for s in s2:
  parts = [ s.part_meronyms(), s.substance_meronyms() ]
  for subset in parts:
    if len(subset) > 0:
      for p in subset:
        print(p.lemma_names())

['accelerator', 'accelerator_pedal', 'gas_pedal', 'gas', 'throttle', 'gun']
['air_bag']
['auto_accessory']
['automobile_engine']
['automobile_horn', 'car_horn', 'motor_horn', 'horn', 'hooter']
['buffer', 'fender']
['bumper']
['car_door']
['car_mirror']
['car_seat']
['car_window']
['fender', 'wing']
['first_gear', 'first', 'low_gear', 'low']
['floorboard']
['gasoline_engine', 'petrol_engine']
['glove_compartment']
['grille', 'radiator_grille']
['high_gear', 'high']
['hood', 'bonnet', 'cowl', 'cowling']
['luggage_compartment', 'automobile_trunk', 'trunk']
['rear_window']
['reverse', 'reverse_gear']
['roof']
['running_board']
['stabilizer_bar', 'anti-sway_bar']
['sunroof', 'sunshine-roof']
['tail_fin', 'tailfin', 'fin']
['third_gear', 'third']
['window']


The other way around, we can also query for "container terms": what concept can contain a specific word as its component?

In [50]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+') # remove punctuation
words = tokenizer.tokenize(midpoint)

for w in words:
  ss = wn.synsets(w)
  for s in ss:
    c = s.member_holonyms()
    if len(c) > 0:
      print(w, c)

people [Synset('world.n.08')]
children [Synset('family.n.02')]
I [Synset('roman_alphabet.n.01')]
s [Synset('roman_alphabet.n.01')]
I [Synset('roman_alphabet.n.01')]


How about concepts that *entail* other concepts?

In [53]:
for w in words:
  ss = wn.synsets(w)
  for s in ss:
    c = s.entailments()
    if len(c) > 0:
      for ss2 in c:
        print(w, ss2.lemma_names())

last ['be', 'live']
see ['look']
walking ['step']
see ['look']
stands ['arise', 'rise', 'uprise', 'get_up', 'stand_up']
playing ['compete', 'vie', 'contend']
saw ['look']
clear ['evaluate', 'pass_judgment', 'judge']
last ['be', 'live']
hold ['procure', 'secure']
have ['perceive', 'comprehend']
have ['conceive']
has ['perceive', 'comprehend']
has ['conceive']


Well, since there are synonym relations (two words with similar meanings), there should naturally also be information on the *antonyms* that carry a contrary meaning,

In [66]:
for w in words:
  if len(w) < 5:
    continue # only process longer words
  ss = wn.synsets(w)
  for s in ss:
    for l in s.lemmas():
      a = l.antonyms()
      if len(a) > 0:
        print(w, a)

vanishing [Lemma('appear.v.02.appear')]
vanishing [Lemma('appear.v.05.appear')]
vague [Lemma('defined.a.01.defined')]
lower [Lemma('raise.v.02.raise')]
lower [Lemma('high.a.01.high')]
lower [Lemma('high.a.02.high')]
lower [Lemma('high.a.04.high')]
walking [Lemma('ride.v.02.ride')]
stands [Lemma('sit.v.01.sit'), Lemma('lie.v.02.lie')]
stands [Lemma('yield.v.05.yield')]
there [Lemma('here.n.01.here')]
there [Lemma('here.r.01.here')]
there [Lemma('here.r.03.here')]
still [Lemma('agitate.v.02.agitate')]
still [Lemma('louden.v.02.louden')]
still [Lemma('moving.a.03.moving')]
still [Lemma('sparkling.a.02.sparkling')]
still [Lemma('no_longer.r.01.no_longer')]
children [Lemma('parent.n.01.parent')]
recall [Lemma('forget.v.02.forget')]
recall [Lemma('issue.v.02.issue')]
bright [Lemma('dull.a.02.dull')]
bright [Lemma('dimmed.a.01.dimmed')]
clear [Lemma('clutter.v.01.clutter')]
clear [Lemma('overcast.v.01.overcast')]
clear [Lemma('bounce.v.04.bounce')]
clear [Lemma('convict.v.01.convict')]
clear 

How can we figure out the *distance* between words? First we have to figure out how deep they are with respect to their root concepts.

In [72]:
words = [ 'chicken', 'dragon', 'princess', 'pie', 'egg' ]
wss = [ wn.synsets(w) for w in words ]
for (w, ss) in zip(words, wss):
  for s in ss:
    print(f'{w} is {s.min_depth()} down from {s.root_hypernyms()}')

chicken is 8 down from [Synset('entity.n.01')]
chicken is 12 down from [Synset('entity.n.01')]
chicken is 5 down from [Synset('entity.n.01')]
chicken is 6 down from [Synset('entity.n.01')]
chicken is 0 down from [Synset('chicken.s.01')]
dragon is 10 down from [Synset('entity.n.01')]
dragon is 7 down from [Synset('entity.n.01')]
dragon is 6 down from [Synset('entity.n.01')]
dragon is 14 down from [Synset('entity.n.01')]
princess is 6 down from [Synset('entity.n.01')]
pie is 7 down from [Synset('entity.n.01')]
pie is 6 down from [Synset('entity.n.01')]
egg is 9 down from [Synset('entity.n.01')]
egg is 6 down from [Synset('entity.n.01')]
egg is 8 down from [Synset('entity.n.01')]
egg is 4 down from [Synset('move.v.02')]
egg is 2 down from [Synset('cover.v.01')]


So, now all the ones that are *entities* should have have a similarity measure that reflects how far apart they are from each other in the hierarchy of meaning.

In [89]:
d = dict(zip(words, wss))
similarities = dict()
for w1 in words:
  ss1 = d[w1]
  for w2 in words:
    if w1 == w2:
      continue
    ss2 = d[w2]
    for (s1, s2) in zip(ss1, ss2): 
      r1 = s1.root_hypernyms()[0]
      r2 = s2.root_hypernyms()[0]
      if r1 == r2:
        sim = s1.path_similarity(s2) # a value in [0, 1], 1 meaning "the same"
        key = (w1, w2, r1)
        value = max(sim, similarities.get(key, 0)) # highest 
        similarities[key] = value

for (w1, w2, root), value in similarities.items():
  print(f'Similarity of {w1} with {w2} is {value} in the hierarchy of {root}')

# extreme cases
print('Highest similarity', max(similarities, key = similarities.get))
print('Lowest similarity', min(similarities, key = similarities.get))

Similarity of chicken with dragon is 0.1111111111111111 in the hierarchy of Synset('entity.n.01')
Similarity of chicken with princess is 0.07692307692307693 in the hierarchy of Synset('entity.n.01')
Similarity of chicken with pie is 0.125 in the hierarchy of Synset('entity.n.01')
Similarity of chicken with egg is 0.08333333333333333 in the hierarchy of Synset('entity.n.01')
Similarity of dragon with chicken is 0.1111111111111111 in the hierarchy of Synset('entity.n.01')
Similarity of dragon with princess is 0.058823529411764705 in the hierarchy of Synset('entity.n.01')
Similarity of dragon with pie is 0.07142857142857142 in the hierarchy of Synset('entity.n.01')
Similarity of dragon with egg is 0.08333333333333333 in the hierarchy of Synset('entity.n.01')
Similarity of princess with chicken is 0.07692307692307693 in the hierarchy of Synset('entity.n.01')
Similarity of princess with dragon is 0.058823529411764705 in the hierarchy of Synset('entity.n.01')
Similarity of princess with pie 